In [1]:
from memory.unsafe import DTypePointer
from memory import memset_zero

In [24]:
struct Field[DT: DType, VW: Int]:
    var data: DTypePointer[DT]
    var gx: Int
    var gy: Int
    var gz: Int
    var dsize: Int
    
    fn __init__(inout self, gx: Int, gy: Int, gz: Int):
        self.dsize = (gx+2)*(gy+2)*(gz+2)
        self.data = DTypePointer[DT].alloc(self.dsize)
        #memset_zero(self.data, self.dsize)
        self.gx = gx
        self.gy = gy
        self.gz = gz

    fn __copyinit__(inout self, other: Self):
        self.gx = other.gx
        self.gy = other.gy
        self.gz = other.gz
        self.data = DTypePointer[DT].alloc(other.dsize)
        self.dsize = other.dsize
        for i in range(0,self.dsize,VW):
            self.data.simd_store[VW](i, other.data.load(i))

    fn __add__(self, rhs: Field[DT,VW]) raises -> Field[DT,VW]:
        # test if the fields have the same sizes
        if self.gx != rhs.gx or self.gy != rhs.gy or self.gz != rhs.gz :
            raise ("cannot add fields with different grid sizes")
        let result = Field[DT,VW](self.gx, self.gy, self.gz)
        let p : Int
        let vwgx = self.gx - self.gx%VW
        for k in range(1,self.gz+1):
            for j in range(1,self.gy+1):
                for i in range (1,vwgx+1,VW):
                    p = k * (self.gx+2) * (self.gy+2) + j * (self.gx+2) + i
                    print (i,j,k,p)
                    result.data.simd_store[VW](p, self.data.simd_load[VW](p) + rhs.data.simd_load[VW](p))
                for i in range (vwgx+1,self.gx+1):
                    p = k * (self.gx+2) * (self.gy+2) + j * (self.gx+2) + i
                    result.data.simd_store[1](p, self.data.simd_load[1](p) + rhs.data.simd_load[1](p))
        return result

    fn __del__(owned self):
        self.data.free()
        print("done del")
        
    fn zero(inout self):
        memset_zero[DT](self.data, self.dsize)

    @always_inline
    fn __getitem__(self, x: Int, y: Int, z: Int) -> SIMD[DT,1]:
        return self.data.load(z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x)

    @always_inline
    fn load[nelts:Int=VW](self, x: Int, y: Int, z: Int) -> SIMD[DT,nelts]:
        return self.data.simd_load[nelts](z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x)

    @always_inline
    fn __setitem__(self, x: Int, y: Int, z: Int, val: SIMD[DT,1]):
        return self.data.store(z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x, val)

    @always_inline
    fn store[nelts:Int=VW](self, x: Int, y: Int, z: Int, val: SIMD[DT, nelts]):
        self.data.simd_store(z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x, val)

In [29]:

let n=8
alias w=1
var a = Field[DType.float32,w](n,1,1)
var b = Field[DType.float32,w](n,1,1)
var c = Field[DType.float32,w](n,1,1)
for i in range(1,n+1):
    a[i,1,1] = 2
    b[i,1,1] = 3
    print(i, a[i,1,1], b[i,1,1])

c = a + b
for i in range(1,n+1):
    print(i, c[i,1,1])

1 2.0 3.0
2 2.0 3.0
3 2.0 3.0
4 2.0 3.0
5 2.0 3.0
6 2.0 3.0
7 2.0 3.0
8 2.0 3.0
1 1 1 41
2 1 1 42
3 1 1 43
4 1 1 44
5 1 1 45
6 1 1 46
7 1 1 47
8 1 1 48
done del
1 5.0
2 5.0
3 5.0
4 5.0
5 5.0
6 5.0
7 5.0
8 5.0


In [21]:
for i in range(2,2):
    print(i)